# Излечение, преобразование, загрузка

## Импортируем необходимые модули

In [10]:
import pandas as pd
from sqlalchemy import create_engine

## Настраиваем подключение к БД

In [11]:
user = "entries_user"
password = "entries_password"
host = "localhost"
port = "5432"
database = "entries_db"

engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")

## intervals_tgt

Читаем сырые данные и справочную таблицу в датафреймы, обрабатывать данные будем в сгруппированном по full_name и отсортированном по event_dt виде, ввиду чего при рассмотрении последовательных пар записей гарантируется упорядоченность.

In [12]:
df_data = pd.read_sql("SELECT * FROM entries_src;", engine, index_col="id")
df_data["event_dt"] = pd.to_datetime(df_data["event_dt"])

df_emergency = pd.read_sql("SELECT * FROM emergency_ref", engine, index_col="id")
df_emergency["event_dt"] = pd.to_datetime(df_emergency["event_dt"])
df_emergency["end_dt"] = df_emergency["event_dt"] + pd.to_timedelta(df_emergency["duration"], unit="m") + pd.Timedelta(minutes=10)

### Очистка от шума

Очищаем данные - убираем дубликаты записей (один и тот же статус, время отличается меньше чем на минуту). После очистки данных возможны следующие пары последовательных статусов:
- **Вход - Выход** - целевой интервал
- **Выход - Вход** - интервал, в который сотрудник не находится на рабочем месте
- **Выход - Доступ запрещён** - аналогично предыдущему пункту, но при начале следующего интервала была попытка неуспешного входа
- **Доступ запрещён - Вход** - Неудачная и удачная попытки входа или вход после интервала, завершившегося статусом **Доступ запрещён**
- **Вход - Доступ запрещён** - Интервал, окончание которого - выход со статусом **Доступ запрещён**
- **Вход - Вход** - Запись о входе после покидания офиса не через турникет (Два случая: запланированно или нет)
- **Доступ запрещён - Доступ запрещён** - интервал, в который сотрудник не находится на рабочем месте, но предыдущий действительный интервал закончился выходом со статусом **Доступ запрещён**, а следующий начинается с попытки неуспешного входа

In [13]:
to_drop = []

for name, group in df_data.groupby("full_name"):
    sorted_group = group.sort_values(by="event_dt")
    rows = list(sorted_group.itertuples(index=True))

    for row_i, row_j in zip(rows, rows[1:]):
        if (row_i.status == row_j.status) and (row_j.event_dt - row_i.event_dt <= pd.Timedelta(seconds=60)):
            to_drop.append(row_i.Index)

df_data.drop(to_drop, inplace=True)

### Пары **Вход - Вход** 

Рассмотрим пару **Вход - Вход**, в обоих случаях первая запись будет валидным началом интервала, для определения второй записи обратимся к справочной таблице **emergency_ref** - если в справочной таблице существует запись о запланированном мероприятии и для времени второй записи выполняется неравенство 
$$event\_dt^{ref} < event\_dt^{src} \leq event\_dt^{ref} + duration^{ref} + 10 (min),$$ 
где 10 минут - учёт очередей на КПП после проведения мероприятия, то мы добавляем запись о выходе, время которой совпадает с временем начала запланированного мероприятия, в противном случае вторую запись о входе удаляем.

In [14]:
to_drop = []
to_push = []

for name, group in df_data.groupby("full_name"):
    sorted_group = group.sort_values(by="event_dt")
    rows = list(sorted_group.itertuples(index=True))

    for row_i, row_j in zip(rows, rows[1:]):        
        if (row_i.status == row_j.status == "Вход"):
            mask_i = (df_emergency["event_dt"] < row_j.event_dt) & (row_j.event_dt < df_emergency["end_dt"])
            mask = (df_emergency["event_dt"] < row_j.event_dt) & (row_j.event_dt < df_emergency["end_dt"])

            if mask.any():
                match = df_emergency[mask]

                to_push.append({
                    "full_name" : name,
                    "event_dt" : match["event_dt"].iloc[0],
                    "status" : "Выход"
                })

            else:
                to_drop.append(row_j.Index)

df_data.drop(to_drop, inplace=True)
df_data = pd.concat([df_data, pd.DataFrame(to_push)])

### Пары **Доступ запрещён - Доступ запрещён**

Рассмотрим пару **Доступ запрещён - Доступ запрещён**, после фильтрации на предыдущем шаге гарантируется, что если записи с индексами $i$-я и $i+1$ - **Доступ запрещён**, то заипси с индексами $i-1$ и $i+2$ - **Вход**, тогда первая запись из пары подразумевает выход, а вторая - ошибочный вход. Соответственно, у первой записи заменяем статус на **Выход**, а вторую удаляем как неуспешный вход.

In [15]:
to_drop = []
to_update = []

for name, group in df_data.groupby("full_name"):
    sorted_group = group.sort_values(by="event_dt")
    rows = list(sorted_group.itertuples(index=True))

    for row_i, row_j in zip(rows, rows[1:]):
        if (row_i.status == row_j.status == "Доступ запрещён"):
            to_drop.append(row_j.Index)
            to_update.append(row_i.Index)

df_data.drop(to_drop, inplace=True)
df_data.loc[to_update, "status"] = "Выход"

### Статус **Доступ запрещён**

Рассмотрим пары, один из элементов которых - **Доступ запрещён**. В первую очередь рассмотрим пару **Выход - Доступ запрещён**, поскольку на первом элементе интервал заканчивается, второй элемент может быть только попыткой неудачного входа, отбросив записи **Доступ запрещён**, получится, что в паре **Доступ запрещён - Вход** появляется однозначность - конец предыдущего интервала и начало нового и значение этой пары становится равным значению пары **Вход - Доступ запрещён**, следовательно все оставшиеся записи **Доступ запрещён** означают выходы и мы можем их заменить.

In [16]:
to_drop = []

for name, group in df_data.groupby("full_name"):
    sorted_group = group.sort_values(by="event_dt")
    rows = list(sorted_group.itertuples(index=True))

    for row_i, row_j in zip(rows, rows[1:]):
        if (row_i.status == "Выход") and (row_j.status == "Доступ запрещён"):
            to_drop.append(row_j.Index)

df_data.drop(to_drop, inplace=True)

df_data.loc[df_data["status"] == "Доступ запрещён", "status"] = "Выход"

### Формирование интервалов

При формировании рассматриваем уникальные пары, при верной обработке на предыдущих шагх гарантируется, что уникальные пары могут иметь вид только **Вход - Выход**. Для рассмотрения случая окончания отчётного периода до окончания рабочего дня обрабатываем непарные записи о входе и присваиваем им крайние дату и время отчётного периода.

In [17]:
report_period_end = df_data["event_dt"].max().normalize() + pd.Timedelta(hours=23, minutes=59, seconds=59)

to_push = []

for name, group in df_data.groupby("full_name"):
    sorted_group = group.sort_values(by="event_dt")
    n = len(sorted_group)

    for i in range(0, n - 1, 2):
        row_i = sorted_group.iloc[i]
        row_j = sorted_group.iloc[i + 1]

        if row_i.status != "Вход" or row_j.status != "Выход":
            print(row_i.status, row_j.status, row_i.event_dt, row_j.event_dt)

        to_push.append({
            "full_name" : row_i.full_name,
            "enter_dt" : row_i.event_dt,
            "exit_dt" : row_j.event_dt
        })

    if n % 2 != 0:
        row = sorted_group.iloc[n - 1]
        to_push.append({
            "full_name" : row.full_name,
            "enter_dt" : row.event_dt,
            "exit_dt" : report_period_end
        })

df_intervals = pd.DataFrame(to_push)

### Выгрузка датафрейма в таблицу и .csv-файл

In [18]:
df_intervals = df_intervals.reset_index()
df_intervals.rename(columns={"index" : "id"}, inplace=True)

df_intervals.to_sql("intervals_tgt", engine, if_exists="append", index=False)
df_intervals.to_csv("data/target/intervals_tgt.csv")